# Indeed Webscrapper 

In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
def get_url(position,location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

# get url from html

In [3]:
url = get_url('business%20analyst', 'United%20States')
print(url)

https://www.indeed.com/jobs?q=business%20analyst&l=United%20States


In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
cards = soup.find_all('div', class_='slider_container')

In [7]:
card = cards[0]
type(card)


bs4.element.Tag

# Test

In [8]:
contents = []
for i in range(0,15):
    contents.append(cards[i].text.strip())
    
contents  

['newBusiness AnalystDatrose3.3Remote$50 - $65 an hourContractEasily applyUrgently hiringHiring multiple candidates\nAbility to analyze data to draw business-relevant conclusions.\nThe individual will work closely with the Global Royalties and Copyright department users,…\nPostedJust posted',
 'newBusiness AnalysteTeam Inc.Irwindale, CA+36 locationsContractEasily apply\nImproves and supports business activities of assigned department through analysis of objectives, goals and customer needs.\nTrack projects in MS Teams/Planner.\nPostedPosted 3 days ago·More...View all eTeam Inc. jobs in Irwindale, CA - Irwindale jobsSalary Search: Business Analyst salaries in Irwindale, CASee popular questions & answers about eTeam Inc.',
 'Junior Business AnalystTalentheed Inc4.6Remote$60,000 - $90,000 a yearFull-time8 hour shiftEasily applyResponsive employerHiring multiple candidates\nMedical coverage qualified following 60 days of business.\nCreating initiatives depending on the business’s requireme

In [9]:
company = card.find('span', 'companyName').text.strip()
company


'Datrose'

In [10]:
job_title = card.find('h2', 'jobTitle').text.strip()
job_title

'newBusiness Analyst'

In [11]:
job_location = card.find('div', 'companyLocation').text.strip()
job_location

'Remote'

In [12]:
post_date = card.find('span', 'date').text.strip()
post_date
today = datetime.today().strftime('%Y-%m-%d')

In [13]:
summary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
summary

'Ability to analyze data to draw business-relevant conclusions. The individual will work closely with the Global Royalties and Copyright department users,…'

In [14]:
if card.find('div',{'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
    salary = card.find('div',{'class':'metadata salary-snippet-container'}).get_text()
elif card.find('class', 'metadata estimated-salary-container'):
    salary = card.find('class', 'metadata estimated-salary-container')
else:
    'Not Available'
salary
    


# salary_tag = card.find('class', 'metadata salary-snippet-container')
# if salary_tag:
#     salary = salary_tag.text.strip()
# else:
#     salary = '' 
# salary_tag

'$50 - $65 an hour'

In [21]:
job_url = 'https://www.indeed.com' + card.find_parent('a').get('href')
job_url

AttributeError: 'NoneType' object has no attribute 'get'

In [15]:
record = (job_title, company, job_location, post_date, today, summary, salary)
record

('newBusiness Analyst',
 'Datrose',
 'Remote',
 'PostedJust posted',
 '2022-06-20',
 'Ability to analyze data to draw business-relevant conclusions. The individual will work closely with the Global Royalties and Copyright department users,…',
 '$50 - $65 an hour')

In [16]:
def get_record(card):
    job_title = card.find('h2', 'jobTitle').text.strip()
    if 'new' in job_title:
        job_title = job_title[3:]
    company = card.find('span', 'companyName').text.strip()
    job_location = card.find('div', 'companyLocation').text.strip()
    post_date = card.find('span', 'date').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
    #job_url = 'https://www.indeed.com' + card.find_parent('a').get('href')
    #salary_tag = card.find('span', 'metadata salary-snippet-container')
    if card.find('div',{'class':'metadata salary-snippet-container'}):
        salary = card.find('div',{'class':'metadata salary-snippet-container'}).get_text()
    elif card.find('class', 'metadata estimated-salary-container'):
        salary = card.find('class', 'metadata estimated-salary-container')
    else:
        salary = 'Not Available'
    record = (job_title, company, job_location, post_date, today, summary, salary)
    return record 

In [17]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)
records

[('Business Analyst',
  'Datrose',
  'Remote',
  'PostedJust posted',
  '2022-06-20',
  'Ability to analyze data to draw business-relevant conclusions. The individual will work closely with the Global Royalties and Copyright department users,…',
  '$50 - $65 an hour'),
 ('Business Analyst',
  'eTeam Inc.',
  'Irwindale, CA+36 locations',
  'PostedPosted 3 days ago',
  '2022-06-20',
  'Improves and supports business activities of assigned department through analysis of objectives, goals and customer needs. Track projects in MS Teams/Planner.',
  'Not Available'),
 ('Junior Business Analyst',
  'Talentheed Inc',
  'Remote',
  'EmployerActive 2 days ago',
  '2022-06-20',
  'Medical coverage qualified following 60 days of business. Creating initiatives depending on the business’s requirements and needs.',
  '$60,000 - $90,000 a year'),
 ('Entry Level Business Analyst',
  'AMGTech Solutions',
  'Chicago, IL',
  'PostedPosted 6 days ago',
  '2022-06-20',
  "Communicate with current and prosp

# Get Next Page

In [19]:
url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
url

'https://www.indeed.com/jobs?q=business+analyst&l=United+States&start=10'

In [20]:
url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'slider_container')

    for card in cards:
        record = get_record(card)
        records.append(record)

In [21]:
#combining codes，run main(position, location)，then saved file could be found in the local file
def get_url(position,location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

def get_record(card):
    job_title = card.find('h2', 'jobTitle').text.strip()
    if 'new' in job_title:
        job_title = job_title[3:]
    company = card.find('span', 'companyName').text.strip()
    #job_url = 'https://www.indeed.com' + card.find_parent('a').get('href')
    job_location = card.find('div', 'companyLocation').text.strip()
    post_date = card.find('span', 'date').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
    if card.find('div',{'class':'metadata salary-snippet-container'}):
        salary = card.find('div',{'class':'metadata salary-snippet-container'}).get_text()
    elif card.find('class', 'metadata estimated-salary-container'):
        salary = card.find('class', 'metadata estimated-salary-container')
    else:
        salary = 'Not Available'
    record = (job_title, company, job_location, post_date, today, summary, salary)
    return record 

def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div','slider_container')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
        
    # Save file
    with open('Todayresults.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary'])
        writer.writerows(records)



In [22]:

main('business analyst','United States')